# Testes com AES-256

## Importação das Bibliotecas

In [ ]:
!pip install pycryptodome cryptography

In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

import base64

# Geração da chave

In [14]:
# Gerar uma chave de 32 bytes (256 bits)
aes_key = get_random_bytes(32)  # Gera uma chave aleatória

## Lista de frases para teste

In [15]:
itens = [
    "a",
    "El Psy Kongroo",
    "1.048596",
    "お前はもう死んでいる.",
    "Para obter algo, é preciso oferecer algo de igual valor."
]

## Funções para criptografar e descriptografar

In [16]:
#  Funções usando PyCryptodome
def encrypt_pycryptodome(text, key):
    iv = get_random_bytes(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(pad(text.encode(), AES.block_size))
    return base64.b64encode(iv + ciphertext).decode()

def decrypt_pycryptodome(ciphertext_b64, key):
    ciphertext = base64.b64decode(ciphertext_b64)
    iv = ciphertext[:16]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ciphertext[16:]), AES.block_size)
    return plaintext.decode()


#  Funções usando cryptography
def encrypt_cryptography(text, key):
    iv = get_random_bytes(16)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(pad(text.encode(), AES.block_size)) + encryptor.finalize()
    return base64.b64encode(iv + ciphertext).decode()

def decrypt_cryptography(ciphertext_b64, key):
    ciphertext = base64.b64decode(ciphertext_b64)
    iv = ciphertext[:16]
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    plaintext = unpad(decryptor.update(ciphertext[16:]) + decryptor.finalize(), AES.block_size)
    return plaintext.decode()


## Execução das funções passando as frases do array

In [ ]:
#  Processar cada item da lista
for item in itens:
    encrypted_py = encrypt_pycryptodome(item, aes_key)
    decrypted_py = decrypt_pycryptodome(encrypted_py, aes_key)

    encrypted_cr = encrypt_cryptography(item, aes_key)
    decrypted_cr = decrypt_cryptography(encrypted_cr, aes_key)

    print("Texto Original:", item)
    print("PyCryptodome:---------------------------------------------------")
    print("Criptografado:", encrypted_py)
    print("Descriptografado:", decrypted_py + "\n")

    print("Cryptography:---------------------------------------------------")
    print("Criptografado:", encrypted_cr)
    print("Descriptografado:", decrypted_cr + "\n")